In [2]:
from cassandra.cluster import Cluster
import pandas as pd

cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sales_data
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

session.set_keyspace('sales_data')

url = 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv'
df = pd.read_csv(url)

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y').dt.date
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y').dt.date

session.execute('DROP TABLE IF EXISTS bronze_sales')

session.execute("""
    CREATE TABLE IF NOT EXISTS bronze_sales (
        region text,
        country text,
        item_type text,
        sales_channel text,
        order_priority text,
        order_date date,
        order_id int PRIMARY KEY,
        ship_date date,
        units_sold int,
        unit_price float,
        unit_cost float,
        total_revenue float,
        total_cost float,
        total_profit float
    )
""")

insert_bronze = session.prepare("""
    INSERT INTO bronze_sales (region, country, item_type, sales_channel, order_priority,
                              order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                              total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for _, row in df.iterrows():
    session.execute(insert_bronze, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))

session.execute('DROP TABLE IF EXISTS silver_sales')

session.execute("""
    CREATE TABLE IF NOT EXISTS silver_sales (
        region text,
        country text,
        item_type text,
        sales_channel text,
        order_priority text,
        order_date date,
        order_id int PRIMARY KEY,
        ship_date date,
        units_sold int,
        unit_price float,
        unit_cost float,
        total_revenue float,
        total_cost float,
        total_profit float
    )
""")

insert_silver = session.prepare("""
    INSERT INTO silver_sales (region, country, item_type, sales_channel, order_priority,
                              order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                              total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for _, row in df.iterrows():
    session.execute(insert_silver, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))

session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_region (
        region text PRIMARY KEY,
        total_sales float
    )
""")

region_sales = df.groupby('Region')['TotalRevenue'].sum()

for region, total_sales in region_sales.items():
    session.execute("""
        INSERT INTO gold_sales_region (region, total_sales)
        VALUES (%s, %s)
    """, (region, total_sales))

session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_country (
        country text PRIMARY KEY,
        total_sales float
    )
""")

country_sales = df.groupby('Country')['TotalRevenue'].sum()

for country, total_sales in country_sales.items():
    session.execute("""
        INSERT INTO gold_sales_country (country, total_sales)
        VALUES (%s, %s)
    """, (country, total_sales))

session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_item_type (
        item_type text PRIMARY KEY,
        total_sales float
    )
""")

item_sales = df.groupby('Item Type')['TotalRevenue'].sum()

for item_type, total_sales in item_sales.items():
    session.execute("""
        INSERT INTO gold_sales_item_type (item_type, total_sales)
        VALUES (%s, %s)
    """, (item_type, total_sales))

rows = session.execute('SELECT * FROM gold_sales_region')
for row in rows:
    print(row)

rows = session.execute('SELECT * FROM gold_sales_country')
for row in rows:
    print(row)

rows = session.execute('SELECT * FROM gold_sales_item_type')
for row in rows:
    print(row)


Row(region='Australia and Oceania', total_sales=10711258.0)
Row(region='Europe', total_sales=34964748.0)
Row(region='Middle East and North Africa', total_sales=24765128.0)
Row(region='Central America and the Caribbean', total_sales=17570836.0)
Row(region='Asia', total_sales=28840812.0)
Row(region='Sub-Saharan Africa', total_sales=24225438.0)
Row(region='North America', total_sales=3611757.5)
Row(country='Malaysia', total_sales=434357.3125)
Row(country='Israel', total_sales=223442.046875)
Row(country='Serbia', total_sales=802989.4375)
Row(country='Djibouti', total_sales=61415.359375)
Row(country='Egypt', total_sales=130261.7578125)
Row(country='Nicaragua', total_sales=5944506.0)
Row(country='Romania', total_sales=1726589.375)
Row(country='Sao Tome and Principe', total_sales=301612.8125)
Row(country='Liberia', total_sales=337990.71875)
Row(country='Vanuatu', total_sales=1222089.25)
Row(country='Morocco', total_sales=503890.09375)
Row(country='Lebanon', total_sales=3699975.25)
Row(country